In [4]:
import pandas as pd
import requests
import time
import csv

# OMDb API settings
api_key = '576b6aa5'  # Replace with your actual API key
base_url_api = 'http://www.omdbapi.com/'

# Load the IMDb movies dataset
csv_file_path = '/Users/mbouch17/Desktop/IronHack/Labs &  Project/Project-3-Week-3/1550_1749_data.csv'
df = pd.read_csv(csv_file_path, encoding='ISO-8859-1')  # Specify encoding

# Extract movie titles from the DataFrame
movie_titles = df['Title'].tolist()  # Adjust the column name as necessary

# Initialize a list to store the fetched movie data
movies_data = []

# Iterate over each movie title in the list and fetch data from the OMDb API
for movie_title in movie_titles:
    params = {
        'apikey': api_key,
        't': movie_title
    }
   
    # Fetch data from OMDb API
    try:
        response = requests.get(base_url_api, params=params)
        response.raise_for_status()  # Raise an error for bad responses
       
        # Print the raw response content for debugging
        print(f"Fetching data for: {movie_title}")
        print("Response text:", response.text)
       
        # Check if the response is empty
        if response.text.strip() == "":
            raise ValueError(f"Empty response for {movie_title}")
       
        # Attempt to parse the JSON response
        movie_data = response.json()
       
    except requests.exceptions.RequestException as e:
        print(f"Network error for {movie_title}: {e}")
        continue
    except ValueError as ve:
        print(f"Error decoding JSON for {movie_title}: {ve}")
        continue
   
    # Check if the movie data is valid
    if movie_data.get('Response') == True:  # The 'Response' field is a string
        # Append relevant data to the movies_data list
        movies_data.append({
               'Title': movie_data.get('Title', None),  # Use get() to avoid KeyError
               'Year': movie_data.get('Year', None),
               'Rated': movie_data.get('Rated', None),
               'Genre': movie_data.get('Genre', None),
               'Runtime': movie_data.get('Runtime', None),
               'Plot': movie_data.get('Plot', None),
               'Actors': movie_data.get('Actors', None),
               'Metascore': movie_data.get('Metascore', None),
               'Director': movie_data.get('Director', None),
               'imdbRating': movie_data.get('imdbRating', None),
               'imdbVotes': movie_data.get('imdbVotes', None),
               'Awards': movie_data.get('Awards', None),
               'Country': movie_data.get('Country', None),
               'Language': movie_data.get('Language', None),
               'BoxOffice': movie_data.get('BoxOffice', None)
           
        })
    elif movie_data.get('Error') == "Request limit reached!":
        print("Request limit reached. Waiting for 5 seconds before retrying...")
        time.sleep(5)  # Wait for 5 seconds before retrying
        continue  # Retry the same title
    else:
        print(f"Error fetching data for '{movie_title}': {movie_data.get('Error')}")
   
    # Optional: Add a delay between requests to avoid hitting rate limits
    time.sleep(2)  # Adjust the delay as needed

# Convert the list of dictionaries into a DataFrame
api_movies_df = pd.DataFrame(movies_data)
df.rename(columns={'Title': 'Title'}, inplace=True)
api_movies_df.rename(columns={'Title': 'Title'}, inplace=True)

if 'Title' in df.columns and 'Title' in api_movies_df.columns:
    df = pd.merge(df, api_movies_df, on='Title', how='inner')
else:
    print("Column 'Title' not found in one of the dataframes")

# Merge the original DataFrame with the API DataFrame
#merged_df = pd.merge(df, api_movies_df, on='Title', how='inner')  # Change to 'outer' if you want to keep all titles

# Save the merged data to a CSV file
output_file = '/Users/mbouch17/Desktop/IronHack/Labs &  Project/Project-3-Week-3/Clean_API/1550_1749_cleaned_movies_data.csv'
df.to_csv(output_file, quoting=csv.QUOTE_ALL, index=False)
print(f"Data saved to {output_file}")


Fetching data for: The Wicker Man
Response text: {"Title":"The Wicker Man","Year":"1973","Rated":"R","Released":"07 Aug 1974","Runtime":"88 min","Genre":"Horror, Mystery, Thriller","Director":"Robin Hardy","Writer":"Anthony Shaffer, David Pinner","Actors":"Edward Woodward, Christopher Lee, Diane Cilento","Plot":"A puritan police sergeant arrives in a Scottish island village in search of a missing girl, who the pagan locals claim never existed.","Language":"English","Country":"United Kingdom","Awards":"3 wins & 6 nominations","Poster":"https://m.media-amazon.com/images/M/MV5BMWU5NmUxMmYtM2QwYS00ODc1LTkxOGItMWY3ZmEzNzJkNGU3XkEyXkFqcGc@._V1_SX300.jpg","Ratings":[{"Source":"Internet Movie Database","Value":"7.5/10"},{"Source":"Rotten Tomatoes","Value":"91%"},{"Source":"Metacritic","Value":"87/100"}],"Metascore":"87","imdbRating":"7.5","imdbVotes":"94,378","imdbID":"tt0070917","Type":"movie","DVD":"N/A","BoxOffice":"$148,882","Production":"N/A","Website":"N/A","Response":"True"}
Error fetch